<a href="https://colab.research.google.com/github/ToniRV/MIT_6.862_Applied_Machine_Learning/blob/master/MIT_6_036_HW04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIT 6.036 Spring 2020: Homework 4
This homework does not include provided Python code. Instead, we
encourage you to write your own code to help you answer some of these
problems, and/or test and debug the code components we do ask for.
Some of the problems below are simple enough that hand calculation
should be possible; your hand solutions can serve as test cases for
your code.  You may also find that including utilities written in
previous labs (like a `sd` or signed distance function) will be
helpful, as you build up additional functions and utilities for
calculation of margins, different loss functions, gradients, and other
functions needed for margin maximization and gradient descent.

In [32]:
!rm -rf code_and_data_for_hw4*
!rm -rf mnist
!wget --quiet https://introml.odl.mit.edu/cat-soop/_static/6.036/homework/hw04/code_and_data_for_hw04.zip
!unzip code_and_data_for_hw04.zip
!mv code_and_data_for_hw04/* .
  
from code_for_hw04 import *
import numpy as np

Archive:  code_and_data_for_hw04.zip
  inflating: code_and_data_for_hw04/code_for_hw04.py  


## 3) Implementing gradient descent
In this section we will implement generic versions of gradient descent and apply these to the logistic regression objective.

<b>Note: </b> If you need a refresher on gradient descent,
you may want to reference
<a href="https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Fall/courseware/Week4/gradient_descent/5">this week's notes</a>.

### 3.1) Implementing Gradient Descent
We want to find the $x$ that minimizes the value of the *objective
function* $f(x)$, for an arbitrary scalar function $f$.  The function
$f$ will be implemented as a Python function of one argument, that
will be a numpy column vector.  For efficiency, we will work with
Python functions that return not just the value of $f$ at $f(x)$ but
also return the gradient vector at $x$, that is, $\nabla_x f(x)$.

We will now implement a generic gradient descent function, `gd`, that
has the following input arguments:

* `f`: a function whose input is an `x`, a column vector, and
  returns a scalar.
* `df`: a function whose input is an `x`, a column vector, and
  returns a column vector representing the gradient of `f` at `x`.
* `x0`: an initial value of $x$, `x0`, which is a column vector.
* `step_size_fn`: a function that is given the iteration index (an
  integer) and returns a step size.
* `num_steps`: the number of iterations to perform

Our function `gd` returns a tuple:

* x: the value at the final step
* fx: the value of f(x) at the final step

**Hint:** This is a short function!

The main function to implement is `gd`, defined below.

In [0]:
def gd(f, df, x0, step_size_fn, num_steps):
    x = x0
    for i in range(num_steps):
        x = x - step_size_fn(i) * df(x)
    return x, f(x)

To evaluate results, we also use a simple `package_ans` function,
which checks the final `x` and `fx` values.

The test cases are provided below, but you should feel free (and are encouraged!) to write more of your own.

In [34]:
test_gd(gd)

Test 1:
Passed!
Test 2:
Passed!


### 3.2) Numerical Gradient
Getting the analytic gradient correct for complicated functions is
tricky.  A very handy method of verifying the analytic gradient or
even substituting for it is to estimate the gradient at a point by
means of *finite differences*.

Assume that we are given a function $f(x)$ that takes a column vector
as its argument and returns a scalar value.  In gradient descent, we
will want to estimate the gradient of $f$ at a particular $x_0.$

The $i^{th}$ component of $\nabla_x f(x_0)$ can be estimated as
$$\frac{f(x_0+\delta^{i}) - f(x_0-\delta^{i})}{2\delta}$$
where $\delta^{i}$ is a column vector whose $i^{th}$ coordinate is
$\delta$, a small constant such as 0.001, and whose other components
are zero.
Note that adding or subtracting $\delta^{i}$ is the same as
incrementing or decrementing the $i^{th}$ component of $x_0$ by
$\delta$, leaving the other components of $x_0$ unchanged.  Using
these results, we can estimate the $i^{th}$ component of the gradient.


**For example**, take $x^(0) = (1,2,3)^T$. The gradient $\nabla_x f(x)$ is a vector of the derivatives of $f(x)$ with respect to each component of $x$, or $\nabla_x f(x) = (\frac{df(x)}{dx_1},\frac{df(x)}{dx_2},\frac{df(x)}{dx_3})^T$.

We can approximate the first component of $\nabla_x f(x)$ as
$$\frac{f((1,2,3)^T+(0.01,0,0)^T) - f((1,2,3)^T-(0.01,0,0)^T)}{2\cdot 0.01}.$$

(We add the transpose so that these are column vectors.)
**This process should be done for each dimension independently,
and together the results of each computation are compiled to give the
estimated gradient, which is $d$ dimensional.**

Implement this as a function `num_grad` that takes as arguments the
objective function `f` and a value of `delta`, and returns a new
**function** that takes an `x` (a column vector of parameters) and
returns a gradient column vector.

**Note:** Watch  out for aliasing. If you do temp_x = x where x is a vector (numpy array), then temp_x is just another name for the same vector as x and changing an entry in one will change an entry in the other. You should either use x.copy() or remember to change entries back after modification.

In [0]:
def num_grad(f, delta=0.001):
    def df(x):
        d = np.zeros(x.shape)
        grad = np.zeros(x.shape)
        for i in range(x.shape[0]):
            d[i] = delta
            grad[i] = (f(x + d) - f(x - d)) / (2 * delta)
            d[i] = 0.0
        return grad
        
    return df

The test cases are shown below; these use the functions defined in the previous exercise.


In [36]:
test_num_grad(num_grad)

Test 1
Passed
Test 2
Passed
Test 3
Passed
Test 4
Passed


A faster (one function evaluation per entry), though sometimes less
accurate, estimate is to use:
$$\frac{f(x_0+\delta^{i}) - f(x_0)}{\delta}$$
for the $i^{th}$ component of $\nabla_x f(x_0).$

### 3.3) Using the Numerical Gradient
Recall that our generic gradient descent function takes both a function
`f` that returns the value of our function at a given point, and `df`,
a function that returns a gradient at a given point.  Write a function
`minimize` that takes only a function `f` and uses this function and
numerical gradient descent to return the local minimum.  We have
provided you with our implementations of `num_grad` and `gd`, so you
should not redefine them in the code box below.
You may use the default of `delta=0.001` for `num_grad`.

**Hint:** Your definition of `minimize` should call `num_grad` exactly
once, to return a function that is called many times.
You should return the same outputs as `gd`.

In [0]:
def minimize(f, x0, step_size_fn, num_steps):
    """
    Parameters:
      See definitions in part 1
    Returns:
      same output as gd
    """
    return gd(f, num_grad(f, delta=0.001), x0, step_size_fn, num_steps)

The test cases are below.

In [38]:
test_minimize(minimize)

Test 1
Passed
Test 2
Passed


## 4) Applying gradient descent to Linear Logistic Classification objective

**Note:** In this section,
you will code many individual functions, each of which depends on previous ones.
We **strongly recommend** that you test each of the components on your own to debug.

### 4.1) Calculating the Linear Logistic Classification (LLC) objective

First, implement the sigmoid function and implement NLL loss over the data points and separator.
Using the latter function, implement the LLC objective.
Note that these functions should work for matrix/vector arguments,
so that we can compute the objective for a whole dataset with one call.

Note that `X` <b>(Upper case X is the dataset here)</b>  is $d \times n$, `y` is $1 \times n$, `th` is $d \times 1$, `th0` is $1 \times 1$, `lam` is a scalar.


In [0]:
# x is a column vector
# returns a vector of the same shape as x
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# X is dxn, y is 1xn, th is dx1, th0 is 1x1
# returns (1,n) the nll loss for each data point given th and th0 
def nll_loss(X, y, th, th0):
    g = sigmoid(np.dot(th.T, X) + th0)
    return -(y * np.log(g) + (1-y) * np.log(1-g))

# X is dxn, y is 1xn, th is dx1, th0 is 1x1, lam is a scalar
# returns (float) the llc objective over the dataset
def llc_obj(X, y, th, th0, lam):
    return 1/y.shape[1] * np.sum(nll_loss(X, y, th, th0)) + lam * np.sum(th**2)

In [93]:
print(sigmoid(np.array([[1, 2], [2, 2]])))

[[0.73105858 0.88079708]
 [0.88079708 0.88079708]]


In [94]:
def super_simple_separable():
    X = np.array([[2, 3, 9, 12],
                  [5, 2, 6, 5]])
    y = np.array([[1, 0, 1, 0]])
    return X, y

sep_e_separator = np.array([[-0.40338351], [1.1849563]]), np.array([[-2.26910091]])

# Test case 1
x_1, y_1 = super_simple_separable()
th1, th1_0 = sep_e_separator
ans = llc_obj(x_1, y_1, th1, th1_0, .1)

print(ans)

# Test case 2
ans = llc_obj(x_1, y_1, th1, th1_0, 0.0)

print(ans)

# Test case beta
sep_e_separator = np.array([[-0.40338351], [1.1849563]]), np.array([[-2.26910091]])
x_1, y_1 = super_simple_separable()
th1, th1_0 = sep_e_separator

0.3739416910005669
0.21725772209560584


In [95]:
print(nll_loss(x_1, y_1, th1, th1_0).tolist())

[[0.0562926158181702, 0.2850112020940751, 0.26093403339997306, 0.266793037070205]]


In [96]:
test_llc_obj(sigmoid,nll_loss,llc_obj)

Test 1 passed
Test 2 passed
Test 3 passed


### 4.2) Calculating the Linear Logistic Classification gradient

Define a function `llc_obj_grad` that returns the gradient of the logistic regression
objective function with respect to $\theta$ and $\theta_0$ in a single
column vector.  The last component of the gradient vector should be
the partial derivative with respect to $\theta_0$.  Look at
`np.vstack` as a simple way of stacking two matrices/vectors
vertically.  We have broken it down into pieces that mimic steps in
the chain rule; this leads to code that is a bit inefficient but
easier to write and debug.  We can worry about efficiency later.

In [0]:
# returns (1,1) the gradient of sigmoid with respect to x
def d_sigmoid(x):
    sx = sigmoid(x)
    return sx*(1-sx)

# returns (d,n) the gradient of nll_loss(X, y, th, th0) with respect to th for each data point
def d_nll_loss_th(X, y, th, th0):
    g = sigmoid(np.dot(th.T, X) + th0)
    return (g - y) * X

# returns (1,n) the gradient of nll_loss(X, y, th, th0) with respect to th0
def d_nll_loss_th0(X, y, th, th0):
    g = sigmoid(np.dot(th.T, X) + th0)
    return (g - y)

# returns (d,1) the gradient of llc_obj(X, y, th, th0) with respect to th
def d_llc_obj_th(X, y, th, th0, lam):
    print(np.sum(d_nll_loss_th(X, y, th, th0), axis=1).reshape(2,1))
    return 1/y.shape[1] * np.sum(d_nll_loss_th(X, y, th, th0), axis=1).reshape(2,1) + 2 * lam * th

# returns (1,1) the gradient of llc_obj(X, y, th, th0) with respect to th0
def d_llc_obj_th0(X, y, th, th0, lam):
    return 1/y.shape[1] * np.array([[np.sum(d_nll_loss_th0(X, y, th, th0))]])

# returns (d+1, 1) the full gradient as a single vector (which includes both th, th0)
def llc_obj_grad(X, y, th, th0, lam):
    return np.vstack((d_llc_obj_th(X, y, th, th0, lam), d_llc_obj_th0(X, y, th, th0, lam)))

In [199]:
# Test d_sigmoid
assert(d_sigmoid(0) == 0.25)
assert(d_sigmoid(100) <= 0.01)
assert(d_sigmoid(-100) <= 0.01)
ans = d_sigmoid(np.array([[ 71, -23.]])).tolist()
expected = [[0.0, 1.0261879629595766e-10]]
if isclose(ans,expected):
    print("Test 1 passed")
else:
    print("Test 1 for d_sigmoid failed")


# Test case d_nll_loss_th
sep_e_separator = np.array([[-0.40338351], [1.1849563]]), np.array([[-2.26910091]])
x_1, y_1 = super_simple_separable()
th1, th1_0 = sep_e_separator
#print(d_nll_loss_th(x_1, y_1, th1, th1_0))

X2, y2 = super_simple_separable()
th2, th20=np.array([[ -3.,  15.]]).T, np.array([[ 2.]])

ans = d_nll_loss_th(X2, y2, th2, th20).tolist()
#print(ans)
expected = [[0.0, 2.9999999996921436, 0.0, 12.0], [0.0, 1.9999999997947624, 0.0, 5.0]]
if isclose(ans,expected):
    print("Test 2 passed")
else:
    print("Test 2 for d_nll_loss_th failed")

ans = d_nll_loss_th0(X2, y2, th2, th20).tolist()
#print(ans)
expected = [[0.0, 0.9999999998973812, 0.0, 1.0]]
if isclose(ans,expected):
    print("Test 3 passed")
else:
    print("Test 3 for d_nll_loss_th0 failed")

ans = d_nll_loss_th(X2, y2, th2, th20).tolist()
#print(ans)
expected = [[0.0, 2.9999999996921436, 0.0, 12.0], [0.0, 1.9999999997947624, 0.0, 5.0]]
if isclose(ans,expected):
    print("Test 2 passed")
else:
    print("Test 2 for d_nll_loss_th failed")

ans = d_nll_loss_th0(X2, y2, th2, th20).tolist()
#print(ans)
expected = [[0.0, 0.9999999998973812, 0.0, 1.0]]
if isclose(ans,expected):
    print("Test 3 passed")
else:
    print("Test 3 for d_nll_loss_th0 failed")


ans = d_llc_obj_th(X2, y2, th2, th20, 0.01).tolist()
print(ans)
expected = [[3.6899999999230357], [2.0499999999486906]]
if isclose(ans,expected):
    print("Test 4 passed")
else:
    print("Test 4 for d_llc_obj_th failed")

ans = d_nll_loss_th0(X2, y2, th2, th20).tolist()
#print(ans)
expected = [[0.0, 0.9999999998973812, 0.0, 1.0]]
if isclose(ans,expected):
    print("Test 5 passed")
else:
    print("Test 5 for d_nll_loss_th0 failed")

ans = d_llc_obj_th0(X2, y2, th2, th20, 0.01).tolist()
#print(ans)
expected = [[0.4999999999743453]]
if isclose(ans,expected):
    print("Test 6 passed")
else:
    print("Test 6 for d_llc_obj_th0 failed")

ans = llc_obj_grad(X2, y2, th2, th20, 0.01).tolist()
#print(ans)
expected = [[3.6899999999230357], [2.0499999999486906], [0.4999999999743453]]
if isclose(ans,expected):
    print("Test 7 passed")
else:
    print("Test 7 for llc_obj_grad failed")


Test 1 passed
Test 2 passed
Test 3 passed
Test 2 passed
Test 3 passed
[[15.]
 [ 7.]]
[[3.6899999999230357], [2.0499999999486906]]
Test 4 passed
Test 5 passed
Test 6 passed
[[15.]
 [ 7.]]
Test 7 passed


Some test cases that may be of use are provided below.

In [200]:
test_llc_grad(d_sigmoid,d_nll_loss_th,d_nll_loss_th0,d_llc_obj_th,d_llc_obj_th0,llc_obj_grad)

Test 1 passed
Test 2 passed
Test 3 passed
[[15.]
 [ 7.]]
Test 4 passed
Test 5 passed
Test 6 passed
[[15.]
 [ 7.]]
Test 7 passed


### 4.3) Linear Logistic Classification minimize

Putting it all together, use the functions you built earlier to write
a gradient descent minimizer for the LLC objective.  You do not need
to paste in your previous definitions; you can just call the ones
you've defined above.  You will need to call `gd`; your function `llc_min` should return
the values that `gd` does.

* Initialize all the separator parameters to zero,
* use the step size function provided below, and
* specify 10 iterations.

In [0]:
def llc_min(data, labels, lam):
    """
    Parameters:
        data: dxn
        labels: 1xn
        lam: scalar
    Returns:
        same output as gd
    """
    def llc_min_step_size_fn(i):
       return 2/(i+1)**0.5
    def f(x):
      th = x[:-1]
      th0 = x[-1]
      X = data
      y = labels
      return llc_obj(X, y, th, th0, lam)
    def df(x):
      th = x[:-1]
      th0 = x[-1]
      X = data
      y = labels
      return llc_obj_grad(X, y, th, th0, lam)
    th = np.zeros((data.shape[0], 1))
    th_0 = np.array([[0.0]])
    x0 = np.vstack((th, th_0))
    assert(x0.shape == (len(th) + 1, 1))
    return gd(f, df, x0, llc_min_step_size_fn, num_steps=10)

Test cases are shown below, where an additional separable test
data set has been specified.

In [233]:
test_llc_min(llc_min)

(2, 1)
(3, 1)
(2, 1)
(1,)
[[ 2.]
 [-2.]]
(2, 1)
(1,)
[[-7.85026201]
 [-5.41013603]]
(2, 1)
(1,)
[[14.99996674]
 [ 6.99997779]]
(2, 1)
(1,)
[[-10.44545813]
 [ -9.62477063]]
(2, 1)
(1,)
[[14.99690995]
 [ 6.99793986]]
(2, 1)
(1,)
[[-9.29749495]
 [-6.75010141]]
(2, 1)
(1,)
[[5.10656093]
 [2.77646576]]
(2, 1)
(1,)
[[-8.68720128]
 [-5.79283976]]
(2, 1)
(1,)
[[14.96570512]
 [ 6.97817016]]
(2, 1)
(1,)
[[-9.00048232]
 [-6.02248214]]
(2, 1)
(1,)
Test 1 passed
(2, 1)
(3, 1)
(2, 1)
(1,)
[[-1.5]
 [ 0.5]]
(2, 1)
(1,)
[[-0.1678709 ]
 [-0.92289946]]
(2, 1)
(1,)
[[-0.39210174]
 [-0.21343158]]
(2, 1)
(1,)
[[-0.37048032]
 [-0.19137596]]
(2, 1)
(1,)
[[-0.35259927]
 [-0.17922816]]
(2, 1)
(1,)
[[-0.33824165]
 [-0.16984708]]
(2, 1)
(1,)
[[-0.32626458]
 [-0.16208368]]
(2, 1)
(1,)
[[-0.31598171]
 [-0.15544939]]
(2, 1)
(1,)
[[-0.30696873]
 [-0.14966325]]
(2, 1)
(1,)
[[-0.2989465 ]
 [-0.14454178]]
(2, 1)
(1,)
Test 2 passed
